# **Feature Engeneering - Bureau**

### **Importar Bibliotecas**

In [1]:
#!/usr/bin/env python
# coding: utf-8

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1701735590419_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import os
import sys
import pytz
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import udf, lpad, translate
from pyspark.sql.types import *
from pyspark.sql.functions import count, avg

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### **Importar tabela 'bureau'.**

In [3]:
spark = SparkSession.builder.appName('Feature Engineering').getOrCreate()
sqlContext=SQLContext(spark.sparkContext)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/mnt/yarn/usercache/livy/appcache/application_1701735590419_0002/container_1701735590419_0002_01_000001/pyspark.zip/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.

In [4]:
file='bureau.csv'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
S3_DATA_SOURCE_PATH = 'pod-academy-analise-de-credito-para-fintech/dados'
S3_DATA_OUTPUT_PATH = 'pod-academy-analise-de-credito-para-fintech/feature-engineering/01-Bureau'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
output = 'abt_bureau'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
full_path_source = 's3://{bkt}/{file}'.format(bkt=S3_DATA_SOURCE_PATH,file=file)
print(full_path_source)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://pod-academy-analise-de-credito-para-fintech/dados/bureau.csv

In [8]:
bureau_00 = spark.read.format(
       "com.databricks.spark.csv").option(
       "header", "true").option(
       "encoding", "ISO-8859-1").option(
       "encoding", "UTF-8").option(
       "inferSchema", "false").option(
        "delimiter", ',').load(
       full_path_source)
bureau_00.registerTempTable("bureau_00")
bureau_00.cache()
qtd=bureau_00.count()
print('records ingestion data:', qtd)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

records ingestion data: 1716428
/mnt/yarn/usercache/livy/appcache/application_1701735590419_0002/container_1701735590419_0002_01_000001/pyspark.zip/pyspark/sql/dataframe.py:330: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.

In [9]:
bureau_00.show(5, truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
|SK_ID_CURR|SK_ID_BUREAU|CREDIT_ACTIVE|CREDIT_CURRENCY|DAYS_CREDIT|CREDIT_DAY_OVERDUE|DAYS_CREDIT_ENDDATE|DAYS_ENDDATE_FACT|AMT_CREDIT_MAX_OVERDUE|CNT_CREDIT_PROLONG|AMT_CREDIT_SUM|AMT_CREDIT_SUM_DEBT|AMT_CREDIT_SUM_LIMIT|AMT_CREDIT_SUM_OVERDUE|CREDIT_TYPE    |DAYS_CREDIT_UPDATE|AMT_ANNUITY|
+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
|215354    |5714462     |Closed       |currency 1     |-497       |0                 |-153.0             |-153.0         

In [10]:
# +----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
# |SK_ID_CURR|SK_ID_BUREAU|CREDIT_ACTIVE|CREDIT_CURRENCY|DAYS_CREDIT|CREDIT_DAY_OVERDUE|DAYS_CREDIT_ENDDATE|DAYS_ENDDATE_FACT|AMT_CREDIT_MAX_OVERDUE|CNT_CREDIT_PROLONG|AMT_CREDIT_SUM|AMT_CREDIT_SUM_DEBT|AMT_CREDIT_SUM_LIMIT|AMT_CREDIT_SUM_OVERDUE|CREDIT_TYPE    |DAYS_CREDIT_UPDATE|AMT_ANNUITY|
# +----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
# |215354    |5714462     |Closed       |currency 1     |-497       |0                 |-153.0             |-153.0           |null                  |0                 |91323.0       |0.0                |null                |0.0                   |Consumer credit|-131              |null       |
# |215354    |5714463     |Active       |currency 1     |-208       |0                 |1075.0             |null             |null                  |0                 |225000.0      |171342.0           |null                |0.0                   |Credit card    |-20               |null       |
# |215354    |5714464     |Active       |currency 1     |-203       |0                 |528.0              |null             |null                  |0                 |464323.5      |null               |null                |0.0                   |Consumer credit|-16               |null       |
# |215354    |5714465     |Active       |currency 1     |-203       |0                 |null               |null             |null                  |0                 |90000.0       |null               |null                |0.0                   |Credit card    |-16               |null       |
# |215354    |5714466     |Active       |currency 1     |-629       |0                 |1197.0             |null             |77674.5               |0                 |2700000.0     |null               |null                |0.0                   |Consumer credit|-21               |null       |
# +----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
bureau_00.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- SK_ID_CURR: string (nullable = true)
 |-- SK_ID_BUREAU: string (nullable = true)
 |-- CREDIT_ACTIVE: string (nullable = true)
 |-- CREDIT_CURRENCY: string (nullable = true)
 |-- DAYS_CREDIT: string (nullable = true)
 |-- CREDIT_DAY_OVERDUE: string (nullable = true)
 |-- DAYS_CREDIT_ENDDATE: string (nullable = true)
 |-- DAYS_ENDDATE_FACT: string (nullable = true)
 |-- AMT_CREDIT_MAX_OVERDUE: string (nullable = true)
 |-- CNT_CREDIT_PROLONG: string (nullable = true)
 |-- AMT_CREDIT_SUM: string (nullable = true)
 |-- AMT_CREDIT_SUM_DEBT: string (nullable = true)
 |-- AMT_CREDIT_SUM_LIMIT: string (nullable = true)
 |-- AMT_CREDIT_SUM_OVERDUE: string (nullable = true)
 |-- CREDIT_TYPE: string (nullable = true)
 |-- DAYS_CREDIT_UPDATE: string (nullable = true)
 |-- AMT_ANNUITY: string (nullable = true)

In [12]:
print(len(bureau_00.columns))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

17

# **Criação de variáveis preditivas**

## **bureau**

Primeiro iremos criar variáveis preditivas na tabela "bureau_balance" para acrescentar essas variáveis na tabela "bureau".

Após essa união das duas tabelas, iremos realizar a criação de variáveis preditivas na tabela "bureau".

In [13]:
# Definir funções de agregações desejadas
agg = ['SUM', 'MAX', 'MIN', 'AVG']

var_categoricas = ['CREDIT_ACTIVE', 'CREDIT_TYPE', 'CREDIT_CURRENCY']

var_numericas = ['CREDIT_DAY_OVERDUE', 'DAYS_CREDIT_ENDDATE',\
                 'AMT_CREDIT_MAX_OVERDUE','AMT_CREDIT_SUM_LIMIT',\
                 'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM', \
                 'AMT_CREDIT_MAX_OVERDUE', 'AMT_ANNUITY']

# Definir flags de meses
# meses_flag = ['ultimos_3_meses_flag', 'ultimos_6_meses_flag', 'ultimos_9_meses_flag', 'ultimos_12_meses_flag']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
bureau_00.createOrReplaceTempView('bureau_00')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## **Criando flags de janela para histórico:**

- últimos 3 meses

- últimos 6 meses

- últimos 12 meses

In [16]:
# bureau_01 = spark.sql("""
# SELECT
#     *,
#     CASE
#         WHEN DAYS_CREDIT BETWEEN MIN(DAYS_CREDIT) OVER (PARTITION BY `SK_ID_BUREAU`) - 90 AND MIN(DAYS_CREDIT) OVER (PARTITION BY `SK_ID_BUREAU`) THEN 1
#         ELSE 0
#     END AS ultimos_3_meses_flag,
#     CASE
#         WHEN DAYS_CREDIT BETWEEN MAX(DAYS_CREDIT) OVER (PARTITION BY `SK_ID_BUREAU`) - 180 AND MAX(DAYS_CREDIT) OVER (PARTITION BY `SK_ID_BUREAU`) THEN 1
#         ELSE 0
#     END AS ultimos_6_meses_flag,
#     CASE
#         WHEN DAYS_CREDIT BETWEEN MAX(DAYS_CREDIT) OVER (PARTITION BY `SK_ID_BUREAU`) - 270 AND MAX(DAYS_CREDIT) OVER (PARTITION BY `SK_ID_BUREAU`) THEN 1
#         ELSE 0
#     END AS ultimos_9_meses_flag,
#     CASE
#         WHEN DAYS_CREDIT BETWEEN MAX(DAYS_CREDIT) OVER (PARTITION BY `SK_ID_BUREAU`) - 360 AND MAX(DAYS_CREDIT) OVER (PARTITION BY `SK_ID_BUREAU`) THEN 1
#         ELSE 0
#     END AS ultimos_12_meses_flag
# FROM bureau_00
# ORDER BY `SK_ID_BUREAU`;
# """)
# bureau_01.createOrReplaceTempView("bureau_01")
# bureau_01.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
# bureau_01.show(5, truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## **Criando variáveis explicativas de primeira camada**

- usar média, mínimo, soma, máximo
- considerar variáveis categóricas e variáveis numéricas

### **Gerar variáveis.**

In [18]:
# Importar SparkSession
from pyspark.sql import SparkSession

# Criar uma sessão do Spark
spark = SparkSession.builder.appName("AdaptacaoCodigo").getOrCreate()

df_original_spark = bureau_00

# Inicializar o contador
contador_resultados = 0

# Criar lista para armazenar os resultados
resultados = []

for ag in agg:
    for var_cat in var_categoricas:
        # Filtrar valores distintos não nulos
        distinct_values = df_original_spark.select(var_cat).distinct().na.drop(subset=[var_cat]).rdd.flatMap(lambda x: x).collect()

        for dom_cat_value in distinct_values:
            for var_numerica in var_numericas:
                resultado = f"ROUND({ag}(CASE WHEN {var_cat} = '{dom_cat_value}' THEN {var_numerica} ELSE NULL END), 2) as {ag.lower()}_{var_numerica.lower().replace('(', '').replace(')', '')}_{var_cat.lower().replace('(', '').replace('-', '').replace(')', '')}_{dom_cat_value.lower().replace(' ', '_').replace('(', '').replace('-', '').replace(')', '')},"

                # Adicionar o resultado à lista
                resultados.append(resultado)

#                 for mes_flag in meses_flag:
#                     resultado = f"ROUND({ag}(CASE WHEN {var_cat} = '{dom_cat_value}' AND {mes_flag} = 1 THEN {var_numerica} ELSE NULL END), 2) as {ag.lower()}_U{mes_flag.split('_')[1]}M_{var_numerica.lower().replace('(', '').replace(')', '')}_{var_cat.lower().replace('(', '').replace('-', '').replace(')', '')}_{dom_cat_value.lower().replace(' ', '_').replace('(', '').replace('-', '').replace(')', '')},"

#                     # Adicionar o resultado à lista
#                     resultados.append(resultado)

                # Incrementar o contador
                contador_resultados += 1

# Printar o total de combinações
print(f"Total de combinações: {contador_resultados}")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total de combina??es: 736

In [19]:
# Inicializa a lista para armazenar os resultados
resultados_formatados = []

# Se há pelo menos um resultado na lista
if resultados:
    # Remove a vírgula do último item
    resultados[-1] = resultados[-1].rstrip(',')

    # Adiciona os resultados formatados à lista
    resultados_formatados.extend(resultados)

    # Une a lista em uma string usando uma quebra de linha como separador
    resultados_str = '\n'.join(resultados_formatados)
    # print(resultados_str)
else:
    resultados_str = "A lista de resultados está vazia."

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### **Colar variáveis criadas no código PySpark-SQL.**

In [20]:
bureau_02 = spark.sql(f'''
SELECT
  SK_ID_CURR as SK_ID_CURR_bureau,
  {resultados_str}
FROM
  bureau_00
GROUP BY
  SK_ID_CURR
''')

bureau_02.createOrReplaceTempView("bureau_02")
bureau_02.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

305811

In [21]:
print(len(bureau_02.columns))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

737

In [29]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

def remove_duplicate_columns(df):
    """
    Remove colunas duplicadas de um DataFrame.

    Parameters:
    - df: DataFrame

    Returns:
    - DataFrame sem colunas duplicadas.
    """
    existing_columns = set()
    columns_to_drop = []

    for column in df.columns:
        if column in existing_columns:
            columns_to_drop.append(column)
        else:
            existing_columns.add(column)

    if columns_to_drop:
        print(f"Colunas duplicadas identificadas e removidas: {columns_to_drop}")
        df = df.drop(*columns_to_drop)

    return df

# Crie uma Spark session
spark = SparkSession.builder.appName("Exemplo").getOrCreate()

# Suponha que 'seu_dataframe' é o seu DataFrame
# Substitua 'seu_dataframe' pelos nomes reais do seu DataFrame
bureau_02 = bureau_02.withColumnRenamed("coluna_duplicada", "nova_coluna_duplicada")

# Use a função para remover colunas duplicadas
bureau_03 = remove_duplicate_columns(bureau_02)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Colunas duplicadas identificadas e removidas: ['sum_amt_credit_max_overdue_credit_active_bad_debt', 'sum_amt_credit_max_overdue_credit_active_active', 'sum_amt_credit_max_overdue_credit_active_closed', 'sum_amt_credit_max_overdue_credit_active_sold', 'sum_amt_credit_max_overdue_credit_type_loan_for_working_capital_replenishment', 'sum_amt_credit_max_overdue_credit_type_real_estate_loan', 'sum_amt_credit_max_overdue_credit_type_unknown_type_of_loan', 'sum_amt_credit_max_overdue_credit_type_loan_for_the_purchase_of_equipment', 'sum_amt_credit_max_overdue_credit_type_cash_loan_nonearmarked', 'sum_amt_credit_max_overdue_credit_type_consumer_credit', 'sum_amt_credit_max_overdue_credit_type_another_type_of_loan', 'sum_amt_credit_max_overdue_credit_type_microloan', 'sum_amt_credit_max_overdue_credit_type_mortgage', 'sum_amt_credit_max_overdue_credit_type_loan_for_business_development', 'sum_amt_credit_max_overdue_credit_type_interbank_credit', 'sum_amt_credit_max_overdue_credit_type_car_loan'

In [30]:
bureau_03.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

305811

In [31]:
print(len(bureau_03.columns))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

553

## **Salvando tabela no diretório do S3.**

In [27]:
# Save to parquet
path_output = os.path.join('s3://',S3_DATA_OUTPUT_PATH,output)
print(path_output)
bureau_03.coalesce(1).write.\
parquet(path_output,mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://pod-academy-analise-de-credito-para-fintech/feature-engineering/01-Bureau/abt_bureau

In [23]:
# abt_bureau=sqlContext.read.format("parquet").load('s3://',S3_DATA_OUTPUT_PATH,output)
# abt_bureau.registerTempTable("abt_bureau")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
# abt_bureau.show(5,truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…